In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,Input,Embedding,Flatten,TextVectorization,Conv1D,GlobalMaxPooling1D,MaxPooling1D,GlobalAveragePooling1D
from keras.initializers import Constant
from keras.layers import Dense,LSTM,Bidirectional,Attention,Concatenate,GRU,BatchNormalization
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
import pickle
import pandas as pd
import jieba

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!mkdir /kaggle/working/data

In [4]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip -d /kaggle/working/data
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip -d /kaggle/working/data
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip -d /kaggle/working/data
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip -d /kaggle/working/data

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: /kaggle/working/data/train.csv  
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: /kaggle/working/data/sample_submission.csv  
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: /kaggle/working/data/test_labels.csv  
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: /kaggle/working/data/test.csv  


In [5]:
data_train = pd.read_csv('/kaggle/working/data/train.csv')
data_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
for column in data_train.columns:
    if column != 'comment_text' and column != 'id' :
        print(data_train[column].value_counts())
        print('-' * 50)

toxic
0    144277
1     15294
Name: count, dtype: int64
--------------------------------------------------
severe_toxic
0    157976
1      1595
Name: count, dtype: int64
--------------------------------------------------
obscene
0    151122
1      8449
Name: count, dtype: int64
--------------------------------------------------
threat
0    159093
1       478
Name: count, dtype: int64
--------------------------------------------------
insult
0    151694
1      7877
Name: count, dtype: int64
--------------------------------------------------
identity_hate
0    158166
1      1405
Name: count, dtype: int64
--------------------------------------------------


In [7]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = data_train[list_classes].values
y_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [8]:
data_test = pd.read_csv('/kaggle/working/data/test.csv')
data_test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
# lowercase and remove extraspace
data_train["comment_text"] = data_train["comment_text"].str.lower()
data_train["comment_text"] = data_train["comment_text"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")
data_test["comment_text"] = data_test["comment_text"].str.lower()
data_test["comment_text"] = data_test["comment_text"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")

In [10]:
data_train['comment_text'][1]

"d'aww! he matches this background colour i'm seemingly stuck with. thanks. (talk) 21:51, january 11, 2016 (utc)"

In [11]:
x_train=data_train['comment_text']
x_test=data_test['comment_text']

In [12]:
label_test=pd.read_csv('/kaggle/working/data/test_labels.csv')
label_test

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [13]:
y_test = label_test[list_classes].values
y_test

array([[-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       ...,
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1]])

In [14]:
y_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [15]:
# Remove common English stop words
def clean(text):
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in text.split() if word not in stop_words]
  text = ' '.join(tokens)
  return text

In [16]:
x_train_cleaned = x_train.apply(clean)
x_test_cleaned = x_test.apply(clean)

In [17]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train_cleaned)
X_train_sequences = tokenizer.texts_to_sequences(x_train_cleaned)
X_test_sequences = tokenizer.texts_to_sequences(x_test_cleaned)

In [18]:
word_index = tokenizer.word_index
total_words = len(word_index)
print(total_words) #same as length of vocabulary

209627


In [19]:
emb_len=len(tokenizer.index_word)+1

In [20]:
#getting the length of the maximum sequence in the dataset
max_length = max([len(w) for w in X_train_sequences])
print(max_length)

1349


In [21]:
# Padding the sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, padding='post', truncating='post')

In [22]:
# Input layer
input_layer = Input(shape=(100,))  # Specify max_len as the maximum sequence length

# Embedding layer
embedding_layer = Embedding(emb_len, 128)(input_layer)

# Bidirectional LSTM layer replaced with Attention layer
lstm_layer = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.3))(embedding_layer)
attention = Attention()([lstm_layer, lstm_layer])  # Attention layer

# 1D Convolutional layer
conv1d_layer = Conv1D(64, kernel_size=3, activation='relu')(attention)

# GlobalMaxPooling1D layer
global_max_pooling_layer = GlobalMaxPooling1D()(conv1d_layer)

# Dense layers
dense_layer_1 = Dense(128, activation='relu')(global_max_pooling_layer)
output_layer = Dense(6, activation='sigmoid')(dense_layer_1)

# Model creation
model = Model(inputs=input_layer, outputs=output_layer)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 128)             2683238   ['input_1[0][0]']             
                                                          4                                       
                                                                                                  
 bidirectional (Bidirection  (None, 100, 256)             263168    ['embedding[0][0]']           
 al)                                                                                              
                                                                                              

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
checkpoint_path = 'training/best_model.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [26]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=True)

In [27]:
#Trainig the model
history = model.fit(X_train_padded,y_train,batch_size=128,epochs=2,validation_split=0.1,callbacks=[model_checkpoint_callback,early_stop])

Epoch 1/2
1122/1122 [==============================] - 899s 792ms/step - loss: 0.0859 - accuracy: 0.9796 - val_loss: 0.0526 - val_accuracy: 0.9937
Epoch 2/2
1122/1122 [==============================] - 808s 721ms/step - loss: 0.0456 - accuracy: 0.9933 - val_loss: 0.0496 - val_accuracy: 0.9940


In [28]:
#Load best model
model.load_weights(checkpoint_path)

In [29]:
## Evaluate Model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

4787/4787 [==============================] - 244s 51ms/step - loss: -3.5054 - accuracy: 0.9989
Validation Loss: -3.5054070949554443, Validation Accuracy: 0.9989162087440491


In [30]:
# Predictions
predictions = model.predict(X_test_padded)
print(predictions)

predictions = pd.DataFrame(predictions)
output = pd.DataFrame(data={'id': data_test['id'],'toxic': predictions[0], 'severe_toxic': predictions[1],
                           'obscene': predictions[2], 'threat': predictions[3], 'insult': predictions[4],
                           'identity_hate': predictions[5]})
output.to_csv('submission.csv', index=False)

4787/4787 [==============================] - 238s 50ms/step
[[9.82801080e-01 1.42222524e-01 8.53312254e-01 5.74658178e-02
  8.42874646e-01 1.49470195e-01]
 [3.01506720e-04 1.72420391e-06 2.93618414e-05 1.42735344e-05
  1.81870855e-05 1.04702031e-05]
 [1.96965225e-02 1.20984776e-04 1.41422672e-03 9.52412607e-04
  2.18868582e-03 9.58185003e-04]
 ...
 [1.07691088e-03 2.51956612e-06 5.44983013e-05 2.73386158e-05
  1.08096756e-04 3.14516619e-05]
 [2.52660806e-03 5.60447825e-06 1.18561526e-04 4.71222120e-05
  2.76020495e-04 6.28822454e-05]
 [9.48032022e-01 7.96377435e-02 6.43577874e-01 6.35010526e-02
  6.83331549e-01 1.39601469e-01]]
